# Lecture 5

 Fall 2023

A demonstration of advanced `pandas` syntax to accompany Lecture 5.

In [123]:
# from google.colab import drive
# drive.mount('/content/drive')

In [124]:
import numpy as np
import pandas as pd
import plotly.express as px

## More on `Groupby`

### Slido Exercise

Try to predict the results of the `groupby` operation shown. The answer is below the image.

<img src="/content/drive/MyDrive/groupby.png" alt="Image" width="600">

The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [125]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [126]:
ds.groupby(ds.index).agg(max)

C:\Users\Maaz PC\AppData\Local\Temp\ipykernel_3584\1820659250.py:1: FutureWarning:

The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.



,x,y
A,3,hi
B,6,tx
C,9,sd


### Loading `babynames` Dataset

In [127]:
import zipfile

# Specify the path to your ZIP file using forward slashes
zip_file_path = "datafiles/babynamesbystate.zip"

# Specify the name of the file you want to read from the ZIP archive
ca_name = 'STATE.CA.TXT'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zf:
    # Check if the specified file exists in the ZIP archive
    if ca_name in zf.namelist():
        # Read the CSV file directly from the ZIP archive
        with zf.open(ca_name) as fh:
            # Define field names for the DataFrame
            field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
            # Read CSV using Pandas
            orgbabynames = pd.read_csv(fh, header=None, names=field_names)
            # Display the first 15 rows of the DataFrame
            # orgbabynames.head(15)
    else:
        print(f"File '{ca_name}' not found in the ZIP archive.")

# orgbabynames.head(15)
babynames = orgbabynames.copy()
babynames.head(10)

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
5,CA,F,1910,Ruth,128
6,CA,F,1910,Evelyn,126
7,CA,F,1910,Alice,118
8,CA,F,1910,Virginia,101
9,CA,F,1910,Elizabeth,93


### Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage in California. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [128]:
# Select the names only. of baby grils
f_babynames = babynames[(babynames['Sex'] == 'F')]

In [129]:
# We sort the data by year
f_babynames = f_babynames[['Name','Year','Count']].sort_values(by='Year')
f_babynames

,Name,Year,Count
0,Mary,1910,295
148,Merle,1910,9
149,Rosalie,1910,9
150,Rosie,1910,9
151,Teresa,1910,9
...,...,...,...
237072,Johana,2022,22
237073,Kit,2022,22
237074,Korra,2022,22
237076,Lavender,2022,22


We'll need a mathematical definition for the change in popularity of a name in California.

Define the metric "Ratio to Peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year.

In [130]:
# Remember that we sorted f_babynames by year.
# This means that grabbing the final entry gives us the most recent count of names
# In 2022, the most recent year for which we have data

We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [131]:
def ratio_to_peak(series):
    return series.iloc[-1] / max(series)

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in the lecture, `pandas` can't apply an aggregation function to non-numeric data (it doens't make sense to divide "CA" by a number). We can select numerical columns of interest directly.

In [132]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

,Year,Count
Name,,
Aadhini,1.0,1.000000
Aadhira,1.0,0.500000
Aadhya,1.0,0.660000
Aadya,1.0,0.586207
Aahana,1.0,0.269231
...,...,...
Zyanya,1.0,0.466667
Zyla,1.0,1.000000
Zylah,1.0,1.000000


In [133]:
# # Results in a TypeError
# rtp_table = f_babynames.groupby("Name").agg(ratio_to_peak)
# rtp_table

### Slido Exercise

Is there a row where `Year` is not equal to 1?

In [134]:
# Find Unique values in the Year column of rtp_table dataframe
rtp_table['Year'].unique()

array([1.])

In [135]:
# Dropping the Year column
rtp_table = rtp_table.drop(columns='Year')
rtp_table

,Count
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.660000
Aadya,0.586207
Aahana,0.269231
...,...
Zyanya,0.466667
Zyla,1.000000
Zylah,1.000000


In [136]:
# Rename "Count" to "Count RTP" for clarity
rtp_table = rtp_table.rename(columns= {'Count' : 'RTP Count'})
rtp_table

,RTP Count
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.660000
Aadya,0.586207
Aahana,0.269231
...,...
Zyanya,0.466667
Zyla,1.000000
Zylah,1.000000


In [137]:
# What name has fallen the most in popularity?
rtp_table = rtp_table.sort_values(by='RTP Count').reset_index()

We can visualize the decrease in the popularity of the name "Debra:"

In [138]:
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)],
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18,
                  width=1000,
                  height=400)
    return fig

plot_name("Debra")

In [139]:
# Find the 10 names that have decreased the most in popularity
top10 = rtp_table.head(10)
top10 = top10['Name']
top10

0      Debra
1     Debbie
2      Carol
3      Tammy
4      Susan
5     Cheryl
6    Shannon
7       Tina
8    Michele
9      Terri
Name: Name, dtype: object

In [140]:
plot_name(*top10)

For fun, try plotting your name or your friends' names.

### Slido Exercise

Given the example below on `babynames` dataset, write code to compute the total number of babies with each name in California using with and without agg.

In [141]:
babies_per_name = babynames.groupby('Name')[['Count']].sum()
babies_per_name

,Count
Name,
Aadan,18
Aadarsh,6
Aaden,647
Aadhav,27
Aadhini,6
...,...
Zymir,5
Zyon,133
Zyra,103


### Slido Exercise

Write code to compute the total number of babies born each year in California.

In [177]:
babies_count = babynames.groupby('Year')[['Count']].sum()
babies_count

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2018,395436
2019,386996
2020,362882


In [178]:
px.line(babies_count, y = "Count")

### `groupby.size` and `groupby.count()`

In [143]:
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [144]:
df.groupby('letter').size()

letter
A    2
B    1
C    3
dtype: int64

`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [145]:
df.groupby('letter').count()


,num,state
letter,,
A,2,1
B,1,1
C,2,2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [146]:
df.groupby('letter').value_counts()


letter  num  state
A       2.0  tx       1
B       3.0  fl       1
C       4.0  ak       1
             hi       1
Name: count, dtype: int64

Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

## Filtering by Group

In [147]:
orgelections = pd.read_csv('datafiles/elections1.csv')
elections = orgelections.copy()
elections.head(10)

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
5,1832,Henry Clay,National Republican,484205,loss,37.603628
6,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
7,1836,Hugh Lawson White,Whig,146109,loss,10.005985
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
9,1836,William Henry Harrison,Whig,550816,loss,37.721543


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [184]:
filtered_election = elections.groupby('Year').filter(lambda x: x['%'].max() < 45)
filtered_election

,Year,Candidate,Party,Popular vote,Result,%
23,1860,Abraham Lincoln,Republican,1855993,win,39.699408
24,1860,John Bell,Constitutional Union,590901,loss,12.639283
25,1860,John C. Breckinridge,Southern Democratic,848019,loss,18.138998
26,1860,Stephen A. Douglas,Northern Democratic,1380202,loss,29.522311
66,1912,Eugene V. Debs,Socialist,901551,loss,6.004354
67,1912,Eugene W. Chafin,Prohibition,208156,loss,1.386325
68,1912,Theodore Roosevelt,Progressive,4122721,loss,27.457433
69,1912,William Taft,Republican,3486242,loss,23.218466
70,1912,Woodrow Wilson,Democratic,6296284,win,41.933422
115,1968,George Wallace,American Independent,9901118,loss,13.571218


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [172]:
# Use agg(max)
elections.groupby('Party').agg(max).head(10)
# NOT CORRECT

C:\Users\Maaz PC\AppData\Local\Temp\ipykernel_3584\2488537279.py:2: FutureWarning:

The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.



,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1976,Thomas J. Anderson,873053,loss,21.554001
American Independent,1976,Lester Maddox,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2016,Michael Peroutka,203091,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,2020,Woodrow Wilson,81268924,win,61.344703


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:

![parties.png](attachment:ab21f8de-ad29-46c2-bea7-e9aea9c40e3e.png)

In [193]:
# one way -- sorting first,group by then picking the first row of every mini df
elections.sort_values(by='%', ascending=False).groupby('Party').first().head(10)

# 2nd -- group by, then sort and pick the first row
# elections.groupby('Party').sort_values(by='%', ascending=False).first().head(10)
# can't do so beacuse no functions can be applied on mini dataframe

# 3rd --

# 4rd -- lamda funtion with iloc(row(0))
elections.sort_values(by='%', ascending=False).groupby('Party').agg(lambda x: x.iloc[0])

# 5th --
elections.loc[elections.groupby("Party")["%"].idxmax()]

# 6th -- using drop duplicates
elections.sort_values("%").drop_duplicates(["Party"], keep="last")

# Another way
def custom_agg(group):
    max_percent = group['%'].max()
    row = group[group['%'] == max_percent].iloc[0]
    return row

elections.groupby('Party').apply(custom_agg).head(10)

C:\Users\Maaz PC\AppData\Local\Temp\ipykernel_3584\4109086473.py:25: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Year,Candidate,Party,Popular vote,Result,%
Party,,,,,,
American,1856,Millard Fillmore,American,873053,loss,21.554001
American Independent,1968,George Wallace,American Independent,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,Anti-Masonic,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,Anti-Monopoly,134294,loss,1.335838
Citizens,1980,Barry Commoner,Citizens,233052,loss,0.270182
Communist,1932,William Z. Foster,Communist,103307,loss,0.261069
Constitution,2008,Chuck Baldwin,Constitution,199750,loss,0.152398
Constitutional Union,1860,John Bell,Constitutional Union,590901,loss,12.639283
Democratic,1964,Lyndon Johnson,Democratic,43127041,win,61.344703


In [194]:
# one way -- sorting first,group by then picking the first row of every mini df
elections.sort_values(by='%', ascending=False).groupby('Party').first().head(10)

,Year,Candidate,Popular vote,Result,%
Party,,,,,
American,1856,Millard Fillmore,873053,loss,21.554001
American Independent,1968,George Wallace,9901118,loss,13.571218
Anti-Masonic,1832,William Wirt,100715,loss,7.821583
Anti-Monopoly,1884,Benjamin Butler,134294,loss,1.335838
Citizens,1980,Barry Commoner,233052,loss,0.270182
Communist,1932,William Z. Foster,103307,loss,0.261069
Constitution,2008,Chuck Baldwin,199750,loss,0.152398
Constitutional Union,1860,John Bell,590901,loss,12.639283
Democratic,1964,Lyndon Johnson,43127041,win,61.344703


#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [ ]:
# Use idxmax function
elections.loc[elections.groupby("Party")["%"].idxmax()]

In [ ]:
# This is the computational part, Extract DataFrame based on above mentioned condition


# Apply sort_index to match the formatting above


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [ ]:
elections.sort_values("%").drop_duplicates(["Party"], keep="last")

*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [205]:
grouped_by_party = elections.groupby("Party")
type(grouped_by_party)
elections

,Year,Candidate,Party,Popular vote,Result,%
0,1824,Andrew Jackson,Democratic-Republican,151271,loss,57.210122
1,1824,John Quincy Adams,Democratic-Republican,113142,win,42.789878
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
3,1828,John Quincy Adams,National Republican,500897,loss,43.796073
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
...,...,...,...,...,...,...
177,2016,Jill Stein,Green,1457226,loss,1.073699
178,2020,Joseph Biden,Democratic,81268924,win,51.311515
179,2020,Donald Trump,Republican,74216154,loss,46.858542
180,2020,Jo Jorgensen,Libertarian,1865724,loss,1.177979


`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [206]:
# visualize groups
grouped_by_party.groups

{'American': [22, 126], 'American Independent': [115, 119, 124], 'Anti-Masonic': [6], 'Anti-Monopoly': [38], 'Citizens': [127], 'Communist': [89], 'Constitution': [160, 164, 172], 'Constitutional Union': [24], 'Democratic': [2, 4, 8, 10, 13, 14, 17, 20, 28, 29, 34, 37, 39, 45, 47, 52, 55, 57, 64, 70, 74, 77, 81, 83, 86, 91, 94, 97, 100, 105, 108, 111, 114, 116, 118, 123, 129, 134, 137, 140, 144, 151, 158, 162, 168, 176, 178], 'Democratic-Republican': [0, 1], 'Dixiecrat': [103], 'Farmer–Labor': [78], 'Free Soil': [15, 18], 'Green': [149, 155, 156, 165, 170, 177, 181], 'Greenback': [35], 'Independent': [121, 130, 143, 161, 167, 174], 'Liberal Republican': [31], 'Libertarian': [125, 128, 132, 138, 139, 146, 153, 159, 163, 169, 175, 180], 'National Democratic': [50], 'National Republican': [3, 5], 'National Union': [27], 'Natural Law': [148], 'New Alliance': [136], 'Northern Democratic': [26], 'Populist': [48, 61, 141], 'Progressive': [68, 82, 101, 107], 'Prohibition': [41, 44, 49, 51, 54,

The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [207]:
grouped_by_party.get_group('Democratic')

,Year,Candidate,Party,Popular vote,Result,%
2,1828,Andrew Jackson,Democratic,642806,win,56.203927
4,1832,Andrew Jackson,Democratic,702735,win,54.574789
8,1836,Martin Van Buren,Democratic,763291,win,52.272472
10,1840,Martin Van Buren,Democratic,1128854,loss,46.948787
13,1844,James Polk,Democratic,1339570,win,50.749477
14,1848,Lewis Cass,Democratic,1223460,loss,42.552229
17,1852,Franklin Pierce,Democratic,1605943,win,51.013168
20,1856,James Buchanan,Democratic,1835140,win,45.306080
28,1864,George B. McClellan,Democratic,1812807,loss,45.048488
29,1868,Horatio Seymour,Democratic,2708744,loss,47.334695


---